# Tensorflow Tutorial

In [17]:
%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf
import skflow
import random
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn import datasets, cross_validation, metrics

random.seed(42)

TensorFlow is all about a building and executing graph. This is a very powerful concept, but it is also cumbersome to start with.
Looking under the hood of Scikit Flow, we just used three parts:
1. TensorFlowTrainer — class that works out all kinks of optimization (builds part of the graph with gradients, does some gradient clipping, and adds an optimizer).
2. logistic_regression — function that creates the graph for logistic regression model.
3. linear_regression — function that creates the graph for linear regression model.
4. DataFeeder — class that samples mini batches of training data into the model (mini batches are needed because training of TensorFlow happens with Stochastic Gradient Descent where random parts of dataset are shown repeatedly to the model).
5. TensorFlowLinearClassifier — this is a class that implements Scikit Learn interface using a LogisticRegression model. It creates a model and a trainer, runs trainer with given dataset on fit() call and runs model in evaluation mode on predict() call.
6. TensorFlowLinearRegressor — similar to TensorFlowClassifier, but uses LinearRegression as a model.

## Multi-layer fully connected neural network
An idea behind TensorFlow (and many other deep learning frameworks) is to be able to connect differentiable parts of the model together and optimize them given the same cost (or loss) function.

In [7]:
data = pd.read_csv('tf_examples/data/titanic_train.csv')
y, X = data['Survived'], data[['Age', 'SibSp', 'Fare']].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
classifier = skflow.TensorFlowDNNClassifier(
...     hidden_units=[10, 20, 10], 
...     n_classes=2, 
...     batch_size=128, 
...     steps=500, 
...     learning_rate=0.05)
>>> classifier.fit(X_train, y_train)
>>> score = accuracy_score(classifier.predict(X_test), y_test)
>>> print("Accuracy: %f" % score)

Step #1, avg. loss: 10.06707
Step #51, epoch #8, avg. loss: 0.73915
Step #101, epoch #16, avg. loss: 0.62324
Step #151, epoch #25, avg. loss: 0.62795
Step #201, epoch #33, avg. loss: 0.62312
Step #251, epoch #41, avg. loss: 0.62120
Step #301, epoch #50, avg. loss: 0.61702
Step #351, epoch #58, avg. loss: 0.61531
Step #401, epoch #66, avg. loss: 0.61496
Step #451, epoch #75, avg. loss: 0.61467
Accuracy: 0.653631


### Multi-layer with tanh activation

In [16]:
>>> def dnn_tanh(X, y):
...    layers = skflow.ops.dnn(X, [10, 20, 10], tf.tanh)
...    return skflow.models.logistic_regression(layers, y)

>>> classifier = skflow.TensorFlowEstimator(
...     model_fn=dnn_tanh, 
...     n_classes=2,
...     batch_size=128,
...     steps=500,
...     learning_rate=0.05)
>>> classifier.fit(X_train, y_train)
>>> score = accuracy_score(classifier.predict(X_test), y_test)
>>> print("Accuracy: %f" % score)

Step #1, avg. loss: 0.68093
Step #51, epoch #8, avg. loss: 0.64784
Step #101, epoch #16, avg. loss: 0.63315
Step #151, epoch #25, avg. loss: 0.63321
Step #201, epoch #33, avg. loss: 0.62441
Step #251, epoch #41, avg. loss: 0.61640
Step #301, epoch #50, avg. loss: 0.61379
Step #351, epoch #58, avg. loss: 0.61022
Step #401, epoch #66, avg. loss: 0.61547
Step #451, epoch #75, avg. loss: 0.61014
Accuracy: 0.692737


As you can see, creating a custom model is as easy as writing a function, that takes X and y inputs (which are Tensors) and returns two tensors: predictions and loss. This is where you can start learning TensorFlow APIs to create parts of sub-graph.

## Digit recognition


In [18]:
digits = datasets.load_digits()
X = digits.images
y = digits.target

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y,
    test_size=0.2, random_state=42)

# TensorFlow model using Scikit Flow ops

def conv_model(X, y):
    X = tf.expand_dims(X, 3)
    features = tf.reduce_max(skflow.ops.conv2d(X, 12, [3, 3]), [1, 2])
    features = tf.reshape(features, [-1, 12])
    return skflow.models.logistic_regression(features, y)

# Create a classifier, train and predict.
classifier = skflow.TensorFlowEstimator(model_fn=conv_model, n_classes=10,
                                        steps=500, learning_rate=0.05,
                                        batch_size=128)
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(classifier.predict(X_test), y_test)
print('Accuracy: %f' % score)

Step #1, avg. loss: 14.36785
Step #51, epoch #4, avg. loss: 4.01558
Step #101, epoch #8, avg. loss: 2.37071
Step #151, epoch #12, avg. loss: 1.73825
Step #201, epoch #16, avg. loss: 1.38814
Step #251, epoch #20, avg. loss: 1.23454
Step #301, epoch #25, avg. loss: 1.12860
Step #351, epoch #29, avg. loss: 1.03399
Step #401, epoch #33, avg. loss: 0.96408
Step #451, epoch #37, avg. loss: 0.89280
Accuracy: 0.652778


## Categorical values

In [19]:
X = data[["Embarked"]]
y = data["Survived"]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)

embarked_classes = X_train["Embarked"].unique()
print('Embarked has next classes: ', embarked_classes)

cat_processor = skflow.preprocessing.CategoricalProcessor()
X_train = np.array(list(cat_processor.fit_transform(X_train)))
X_test = np.array(list(cat_processor.transform(X_test)))

# Total number of classes for this variable from Categorical Processor.
# Includes unknown token and unique classes for variable.
n_classes = len(cat_processor.vocabularies_[0])

### Embeddings

EMBEDDING_SIZE = 3

def categorical_model(X, y):
    features = skflow.ops.categorical_variable(
        X, n_classes, embedding_size=EMBEDDING_SIZE, name='embarked')
    return skflow.models.logistic_regression(tf.squeeze(features, [1]), y)

classifier = skflow.TensorFlowEstimator(model_fn=categorical_model,
    n_classes=2)
classifier.fit(X_train, y_train)

print("Accuracy: {0}".format(metrics.accuracy_score(classifier.predict(X_test), y_test)))
print("ROC: {0}".format(metrics.roc_auc_score(classifier.predict(X_test), y_test)))

### One Hot

def one_hot_categorical_model(X, y):
    features = skflow.ops.one_hot_matrix(X, n_classes)
    return skflow.models.logistic_regression(tf.squeeze(features, [1]), y)

classifier = skflow.TensorFlowEstimator(model_fn=one_hot_categorical_model,
    n_classes=2, steps=1000, learning_rate=0.01)
classifier.fit(X_train, y_train)

print("Accuracy: {0}".format(metrics.accuracy_score(classifier.predict(X_test), y_test)))
print("ROC: {0}".format(metrics.roc_auc_score(classifier.predict(X_test), y_test)))

('Embarked has next classes: ', array(['S', 'C', 'Q', nan], dtype=object))
Step #1, avg. loss: 0.76733
Step #21, avg. loss: 0.67197
Step #41, epoch #1, avg. loss: 0.67355
Step #61, epoch #2, avg. loss: 0.65313
Step #81, epoch #3, avg. loss: 0.65903
Step #101, epoch #4, avg. loss: 0.65579
Step #121, epoch #5, avg. loss: 0.64565
Step #141, epoch #6, avg. loss: 0.65923
Step #161, epoch #7, avg. loss: 0.65823
Step #181, epoch #7, avg. loss: 0.65129
Accuracy: 0.625698324022
ROC: 0.610550615595
Step #1, avg. loss: 0.82722
Step #101, epoch #4, avg. loss: 0.82588
Step #201, epoch #8, avg. loss: 0.68599
Step #301, epoch #13, avg. loss: 0.66034
Step #401, epoch #17, avg. loss: 0.65153
Step #501, epoch #21, avg. loss: 0.65113
Step #601, epoch #26, avg. loss: 0.65095
Step #701, epoch #30, avg. loss: 0.65017
Step #801, epoch #34, avg. loss: 0.65184
Step #901, epoch #39, avg. loss: 0.64966
Accuracy: 0.625698324022
ROC: 0.610550615595
